In [1]:
import os
import random
import numpy as np
import pandas as pd 
from skimage import io
from skimage import color
from PIL import Image
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
from dask.array.image import imread
from dask import bag, threaded
from dask.diagnostics import ProgressBar
import cv2
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")



import keras
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense,GlobalAveragePooling2D
from keras.layers import Flatten,Dropout
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image 
from keras.layers.normalization import BatchNormalization
from keras import optimizers


Using TensorFlow backend.


In [2]:

## Getting all the images

val_image = []


imgs = sorted(os.listdir('/home/jupyter/Test_Images/test'))

cntr = 0
l = 0

for j in range(len(imgs)):
    img_name = "/home/jupyter/Test_Images/test"+"/"+imgs[j]
    img = cv2.imread(img_name)
    #img = color.rgb2gray(img)
    img = img[50:,120:-50]
    img = cv2.resize(img,(224,224))
    val_image.append(img)
    cntr = cntr+1
    if cntr == 1000:
        l = l+1
        print ('Number of images read:',l*1000)
        cntr = 0

print('All images are loaded')


Number of images read: 1000
Number of images read: 2000
Number of images read: 3000
Number of images read: 4000
Number of images read: 5000
Number of images read: 6000
Number of images read: 7000
Number of images read: 8000
Number of images read: 9000
Number of images read: 10000
Number of images read: 11000
Number of images read: 12000
Number of images read: 13000
Number of images read: 14000
Number of images read: 15000
Number of images read: 16000
Number of images read: 17000
Number of images read: 18000
Number of images read: 19000
Number of images read: 20000
Number of images read: 21000
Number of images read: 22000
Number of images read: 23000
Number of images read: 24000
Number of images read: 25000
Number of images read: 26000
Number of images read: 27000
Number of images read: 28000
Number of images read: 29000
Number of images read: 30000
Number of images read: 31000
Number of images read: 32000
Number of images read: 33000
Number of images read: 34000
Number of images read: 

In [3]:
## Splitting the train and test

X_test = []

for features in val_image:
    X_test.append(features)
    

print (len(X_test))

79726


In [4]:
## Converting images to nparray. Encoding the Y

X_test = np.array(X_test).reshape(-1,224,224,3)

print (X_test.shape)

(79726, 224, 224, 3)


### Exception Model: Extra Layers - Accuracy 82.5% Loss 0.52

In [5]:
## Defining the input

from keras.layers import Input
xception_input = Input(shape = (224, 224, 3), name = 'Image_input')

## The RESNET model

from keras.applications.xception import preprocess_input, decode_predictions
from keras.applications.xception import Xception


#Get the RESNET weights and layers

model_xception_conv = Xception(weights= 'imagenet', include_top=False, input_shape= (224,224,3))

from keras.models import Model


output_xception_conv = model_xception_conv(xception_input)

#Add the fully-connected layers 

x=GlobalAveragePooling2D()(output_xception_conv)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x = Dropout(0.1)(x) # **reduce dropout 
x=Dense(1024,activation='relu')(x) #dense layer 2
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(512,activation='relu')(x) #dense layer 3
x = Dense(10, activation='softmax', name='predictions')(x)


xception_pretrained = Model(input = xception_input, output = x)

xception_pretrained = Model(input = xception_input, output = x)
xception_pretrained.load_weights('xception_weights_aug_extralayer_alltrained_sgd2_V2.hdf5')

print('Model is Loaded')


Model is Loaded


In [6]:
# labels is the image array

from sklearn.metrics import accuracy_score, confusion_matrix

model1_prediction = []
model1_pred_class = []

model1_prediction = xception_pretrained.predict(X_test)
print('Images Predicted until now:',len(model1_prediction))



Images Predicted until now: 79726


### Exception Model: No Extra Layers - Accuracy 82.8% Loss 0.55

In [7]:
## Defining the input

from keras.layers import Input
xception_input = Input(shape = (224, 224, 3), name = 'Image_input')

## The RESNET model

from keras.applications.xception import preprocess_input, decode_predictions
from keras.applications.xception import Xception


#Get the RESNET weights and layers

model_xception_conv = Xception(weights= 'imagenet', include_top=False, input_shape= (224,224,3))

from keras.models import Model


output_xception_conv = model_xception_conv(xception_input)

#Add the fully-connected layers 

x = Flatten(name='flatten')(output_xception_conv)
x = Dense(10, activation='softmax', name='predictions')(x)


xception_pretrained = Model(input = xception_input, output = x)

xception_pretrained = Model(input = xception_input, output = x)
xception_pretrained.load_weights('xception_weights_aug_alltrained_setval_sgd3.hdf5')

print('Model is Loaded')

Model is Loaded


In [8]:
# labels is the image array

model2_prediction = []
model2_pred_class = []

model2_prediction = xception_pretrained.predict(X_test)
print('Images Predicted until now:',len(model2_prediction))



Images Predicted until now: 79726


### RESNET 50 Model: No Extra Layers - Accuracy 85.43% Loss 0.55

In [9]:
## Defining the input
from keras.layers import Input
from keras.models import Model
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.applications.resnet50 import ResNet50


resnet50_input = Input(shape = (224, 224, 3), name = 'Image_input')
model_resnet50_conv = ResNet50(weights= 'imagenet', include_top=False, input_shape= (224,224,3))

output_resnet50_conv = model_resnet50_conv(resnet50_input)

x = Flatten(name='flatten')(output_resnet50_conv)
x = Dense(10, activation='softmax', name='predictions')(x)
resnet50_pretrained = Model(input = resnet50_input, output = x)

resnet50_pretrained.load_weights('resnet_weights_aug_alltrained_sgd2_setval.hdf5')

print('Model is Loaded')



Model is Loaded


In [10]:
# labels is the image array


model3_prediction = []
model3_pred_class = []

model3_prediction = resnet50_pretrained.predict(X_test)
print('Images Predicted until now:',len(model3_prediction))



Images Predicted until now: 79726


### RESNET 50 Model: Extra Layers - Accuracy 86.43% Loss 0.47

In [11]:
## Defining the input

from keras.layers import Input
from keras.models import Model
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.applications.resnet50 import ResNet50

resnet50_input = Input(shape = (224, 224, 3), name = 'Image_input')

## The RESNET model
model_resnet50_conv = ResNet50(weights= 'imagenet', include_top=False, input_shape= (224,224,3))



output_resnet50_conv = model_resnet50_conv(resnet50_input)
x=GlobalAveragePooling2D()(output_resnet50_conv)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x = Dropout(0.1)(x) # **reduce dropout 
x=Dense(1024,activation='relu')(x) #dense layer 2
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x=Dense(512,activation='relu')(x) #dense layer 3
x = Dense(10, activation='softmax', name='predictions')(x)


resnet50_pretrained = Model(input = resnet50_input, output = x)

resnet50_pretrained.load_weights('resnet_weights_aug_extralayers_sgd_setval.hdf5')


print('Model is loaded')


Model is loaded


In [12]:
# labels is the image array


model4_prediction = []
model4_pred_class = []

model4_prediction = resnet50_pretrained.predict(X_test)
print('Images Predicted until now:',len(model4_prediction))


Images Predicted until now: 79726


### VGG 16 Model: No Extra Layers - Accuracy 84.7% Loss 0.5

In [13]:
## Defining the input

from keras.layers import Input
from keras.models import Model


vgg16_input = Input(shape = (224, 224, 3), name = 'Image_input')

from keras.applications.vgg16 import VGG16, preprocess_input

model_vgg16_conv = VGG16(weights='imagenet', include_top=False, input_tensor = vgg16_input)



output_vgg16_conv = model_vgg16_conv(vgg16_input)

x = Flatten(name='flatten')(output_vgg16_conv)
x = Dense(10, activation='softmax', name='predictions')(x)

vgg16_pretrained = Model(input = vgg16_input, output = x)


vgg16_pretrained.load_weights('vgg_weights_aug_setval_sgd.hdf5')

print('The Model is loaded')

The Model is loaded


In [14]:
# labels is the image array


model5_prediction = []
model5_pred_class = []

model5_prediction = vgg16_pretrained.predict(X_test)
print('Images Predicted until now:',len(model5_prediction))


Images Predicted until now: 79726


### VGG 16 Model: Extra Layers - Accuracy 81.5% Loss 0.57

In [15]:
## Defining the input

from keras.layers import Input
from keras.models import Model


vgg16_input = Input(shape = (224, 224, 3), name = 'Image_input')

from keras.applications.vgg16 import VGG16, preprocess_input

model_vgg16_conv = VGG16(weights='imagenet', include_top=False, input_tensor = vgg16_input)



output_vgg16_conv = model_vgg16_conv(vgg16_input)

x=GlobalAveragePooling2D()(output_vgg16_conv)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x = Dropout(0.1)(x) # **reduce dropout 
x=Dense(1024,activation='relu')(x) #dense layer 2
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
x = Dense(512,activation='relu')(x) #dense layer 3
x = Dense(10, activation='softmax', name='predictions')(x)

vgg16_pretrained = Model(input = vgg16_input, output = x)


vgg16_pretrained.load_weights('vgg_weights_aug_setval_layers_sgd2.hdf5')

print('The Model is loaded')

The Model is loaded


In [16]:
# labels is the image array


model6_prediction = []
model6_pred_class = []

model6_prediction = vgg16_pretrained.predict(X_test)
print('Images Predicted until now:',len(model6_prediction))



Images Predicted until now: 79726


### Ensembling

In [24]:
from statistics import mean,median
predictions = []
ensemble_predictions = []

for i in range(len(X_test)):
#for i in range(1):
    mean_prediction = []
    
    for j in range(10):
        predictions.append(model1_prediction[i][j])
        predictions.append(model2_prediction[i][j])
        predictions.append(model3_prediction[i][j])
        predictions.append(model4_prediction[i][j])
        predictions.append(model5_prediction[i][j])
        predictions.append(model6_prediction[i][j])
        #print(predictions)
        
        trimmed_value = (sum(predictions) - max(predictions) - min(predictions))/(len(predictions) - 2)
        
        mean_value = mean(predictions)

        predictions = []
        mean_prediction.append(trimmed_value)
        #mean_prediction.append(mean_value)
    
    mean_prediction = mean_prediction/ sum(mean_prediction)
    ensemble_predictions.append(mean_prediction)
        
    
#ensemble_predictions = np.asarray(ensemble_predictions)        
print('Got predictions from ensemble')
    

Got predictions from ensemble


In [25]:
df_pred = pd.DataFrame(ensemble_predictions)

In [26]:
df_pred['Images'] = pd.Series(imgs)

In [27]:
df_pred.to_csv('EnsemblePredictions_2.csv')

In [23]:


# df_pred = pd.DataFrame(model6_prediction)
# df_pred['Images'] = pd.Series(imgs)
# df_pred.to_csv('Model6_Predictions.csv')